In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import string
import itertools
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
from Levenshtein import distance as lev
from ast import literal_eval
import pickle
from sklearn.model_selection import KFold
import fasttext

In [ ]:
noteEvents=pd.read_csv("mimic-iii-clinical-database-1.4/NOTEEVENTS.csv")
diagnosis = pd.read_csv("mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv")

C:\Users\balle\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Preprocessing

In [ ]:
noteEvents=noteEvents[["HADM_ID","TEXT","DESCRIPTION"]]
diagnosis = diagnosis[["HADM_ID","SUBJECT_ID","ICD9_CODE"]]

def exclude_procedure_code(x):
    if str(x).startswith("0"):
        return False
    return True

#Group Diagnosis into 1 list per admission
diagnosis=diagnosis[diagnosis["ICD9_CODE"].apply(lambda x: exclude_procedure_code(x))]

In [ ]:
def convert_icd9(x):
    converted_code = []
    for code in x:
        icd9_str = str(code)
        if icd9_str.startswith("E"):
            converted = icd9_str[:4]
        else:
            converted = icd9_str[:3]
        converted_code.append(converted)
    return converted_code

def checkdiagnosis(x):
    for code in x:
        if str(code)[0:3]=="250":
            return True
    return False

def exclude_procedure_code(x):
    included_code = []
    for code in x:
        icd9_str = str(code)
        if icd9_str.startswith("0"):
            pass
        else:
            included_code.append(code)
    return included_code

In [ ]:
diagnosisGrouped = diagnosis.groupby(["SUBJECT_ID","HADM_ID"])["ICD9_CODE"].apply(list).reset_index()
diagnosisGrouped.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,2,163353,"[V3001, V053, V290]"
1,3,145834,"[78559, 5849, 4275, 41071, 4280, 6826, 4254, 2..."
2,4,185777,"[1363, 7994, 2763, 7907, 5715, V090, E9317]"
3,5,178980,"[V3000, V053, V290]"
4,6,107064,"[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1..."


In [ ]:
diagnosisGrouped["ICD9_CODE"] = diagnosisGrouped["ICD9_CODE"].apply(lambda x: exclude_procedure_code(x))
admissionswithDiabetes = diagnosisGrouped[diagnosisGrouped["ICD9_CODE"].apply(lambda x: checkdiagnosis(x))]
admissionswithDiabetes["ICD9_CODE_primary"] = admissionswithDiabetes["ICD9_CODE"].apply(lambda x: convert_icd9(x))
admissionswithDiabetes.head()

C:\Users\balle\AppData\Local\Temp/ipykernel_8176/642132842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissionswithDiabetes["ICD9_CODE_primary"] = admissionswithDiabetes["ICD9_CODE"].apply(lambda x: convert_icd9(x))


,SUBJECT_ID,HADM_ID,ICD9_CODE,ICD9_CODE_primary
11,13,143045,"[41401, 4111, 25000, 4019, 2720]","[414, 411, 250, 401, 272]"
15,18,188822,"[25080, 78039, 29633, V5867, E9323, V5869, 478...","[250, 780, 296, V58, E932, V58, 478, 780, 783,..."
17,20,157681,"[41401, 4111, 25000, 2724, 4019]","[414, 411, 250, 272, 401]"
18,21,109451,"[41071, 78551, 5781, 5849, 40391, 4280, 4592, ...","[410, 785, 578, 584, 403, 428, 459, 507, 427, ..."
19,21,111970,"[78552, 40391, 42731, 70709, 5119, 6823, 99859...","[785, 403, 427, 707, 511, 682, 998, 572, 995, ..."


In [ ]:
combineddf=noteEvents.merge(admissionswithDiabetes, on="HADM_ID")
print(combineddf.shape)
combineddf.head()

(406203, 6)


,HADM_ID,TEXT,DESCRIPTION,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
0,121936.0,Admission Date: [**2125-2-9**] D...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
1,121936.0,PATIENT/TEST INFORMATION:\nIndication: Aortic ...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
2,121936.0,PATIENT/TEST INFORMATION:\nIndication: Aortic...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
3,121936.0,Sinus rhythm. Frequent atrial premature beats...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
4,121936.0,Rhythm is most likely sinus rhythm with freque...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."


In [ ]:
processeddf=combineddf.copy()
spacestring=" "*len(string.punctuation.replace("'",""))
dstring="d"*len(string.digits)
processeddf["TEXT"]=processeddf["TEXT"].apply(lambda x: x.translate(str.maketrans(string.punctuation.replace("'",""),spacestring)))
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.translate(str.maketrans(string.digits,dstring)))
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.lower())
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.split())

In [ ]:
wordslist=processeddf["TEXT"].tolist()
flatlist=list(itertools.chain(*wordslist))
wordset=set(flatlist)
wordDict=Counter(flatlist)
wordDict5orMore= dict(filter(lambda x: x[1] >=5, wordDict.items())) 

wordDictLessThan5 = dict(filter(lambda x: x[1] <5, wordDict.items()))

In [ ]:
print(len(wordDict5orMore)) #53229
print(len(wordDictLessThan5)) #109486

53229
109486


In [ ]:
###GENERATE Dictionary for mapping mispelled words
# mapped_dict = dict()

# for w in range(len(wordListLessThan5)):
#     lev_dist = []
#     for i in range(len(wordList5orMore)):
#         lev_dist.append(lev(wordListLessThan5[w], wordList5orMore[i]))
#     print(np.argmin(lev_dist))
#     mapped_dict[wordListLessThan5[w]] = wordList5orMore[np.argmin(lev_dist)]

# print(len(mapped_dict))

# with open('mapDictionary.pkl', 'wb') as f:
#     pickle.dump(mapped_dict, f)

with open('mapDictionary.pkl', 'rb') as f:
    mapped_dict = pickle.load(f)

In [ ]:
limited_processeddf = processeddf[processeddf['TEXT'].apply(lambda x: len(x)) > 9]
limited_processeddf = limited_processeddf[limited_processeddf['TEXT'].apply(lambda x: len(x)) < 2200]
print(limited_processeddf.shape)
limited_processeddf.head()


(399623, 6)


,HADM_ID,TEXT,DESCRIPTION,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
1,121936.0,"[patient, test, information, indication, aorti...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
2,121936.0,"[patient, test, information, indication, aorti...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
3,121936.0,"[sinus, rhythm, frequent, atrial, premature, b...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
4,121936.0,"[rhythm, is, most, likely, sinus, rhythm, with...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
5,121936.0,"[atrial, fibrillation, intraventricular, condu...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."


In [ ]:
def map_misspelled(x):
    new_l = [mapped_dict.get(item, item) for item in x]
    return new_l

limited_processeddf["TEXT"] = limited_processeddf["TEXT"].apply(lambda x: map_misspelled(x))

In [ ]:
icd9_list=limited_processeddf["ICD9_CODE_primary"].tolist()
icd9_flatlist=list(itertools.chain(*icd9_list))
print("Num. icd9 primary code",len(icd9_flatlist))
icd9_wordset=set(icd9_flatlist)
icd9_wordDict=Counter(icd9_flatlist)
print("Num. unique icd9 primary code",len(icd9_wordDict))
icd9_re_list=limited_processeddf["ICD9_CODE"].tolist()
icd9_re_flatlist=list(itertools.chain(*icd9_re_list))
print("Num. icd9 regular code",len(icd9_re_flatlist))
icd9_re_wordset=set(icd9_re_flatlist)
icd9_re_wordDict=Counter(icd9_re_flatlist)
print("Num. unique icd9 regular code",len(icd9_re_wordDict))

Num. icd9 primary code 6703628
Num. unique icd9 primary code 808
Num. icd9 regular code 6703628
Num. unique icd9 regular code 3965


In [ ]:
limited_processeddf["ICD9_SET"]= limited_processeddf["ICD9_CODE"].apply(lambda x: set(x))
limited_processeddf["NumberofICD9Codes"] = limited_processeddf["ICD9_SET"].apply(lambda x: len(x))
limited_processeddf["NumberofICD9CodesPri"] = limited_processeddf["ICD9_CODE_primary"].apply(lambda x: len(set(x)))
CodesperReport = limited_processeddf["NumberofICD9Codes"].tolist()
avgcodesperReport = sum(CodesperReport) / len(CodesperReport)
PriCodesperReportlist = limited_processeddf["NumberofICD9CodesPri"].tolist()
avgPriCodesperReport = sum(PriCodesperReportlist) / len(PriCodesperReportlist)
print("avgCodesperReport",avgcodesperReport)
print("avgPriCodesperReport",avgPriCodesperReport)

avgCodesperReport 16.773669183205172
avgPriCodesperReport 15.200246232073729


In [ ]:
limited_processeddf["NumberofTokens"] = limited_processeddf["TEXT"].apply(lambda x: len(x))
NumberofTokensList=limited_processeddf["NumberofTokens"].tolist()
avgTokensperReport = sum(NumberofTokensList) / len(NumberofTokensList)
print("avgTokensperReport",avgTokensperReport)

avgTokensperReport 309.05638814582744


In [ ]:
print("avgDensityperReport",avgcodesperReport/len(icd9_wordDict))
print("avgPriDensityperReport",avgPriCodesperReport/len(icd9_re_wordDict))

avgDensityperReport 0.02075949156337274
avgPriDensityperReport 0.0038336056070803854


# Model

In [ ]:
textfiledf=limited_processeddf[["TEXT","ICD9_CODE"]]

def stringText(lst):
    return ' '.join(lst)

textfiledf["ReportTokens"]=textfiledf["TEXT"].apply(lambda x: stringText(x))


C:\Users\balle\AppData\Local\Temp/ipykernel_8176/3641008178.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textfiledf["ReportTokens"]=textfiledf["TEXT"].apply(lambda x: stringText(x))


In [ ]:
def Labels(lst):
    string=' __label__'.join(lst)
    return "__label__"+string

textfiledf["Labels"]=textfiledf["ICD9_CODE"].apply(lambda x: Labels(x))
textfiledf.head(5)


C:\Users\balle\AppData\Local\Temp/ipykernel_8176/2917634657.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textfiledf["Labels"]=textfiledf["ICD9_CODE"].apply(lambda x: Labels(x))


,TEXT,ICD9_CODE,ReportTokens,Labels
1,"[patient, test, information, indication, aorti...","[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...",patient test information indication aortic val...,__label__42843 __label__41071 __label__5990 __...
2,"[patient, test, information, indication, aorti...","[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...",patient test information indication aortic val...,__label__42843 __label__41071 __label__5990 __...
3,"[sinus, rhythm, frequent, atrial, premature, b...","[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...",sinus rhythm frequent atrial premature beats c...,__label__42843 __label__41071 __label__5990 __...
4,"[rhythm, is, most, likely, sinus, rhythm, with...","[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...",rhythm is most likely sinus rhythm with freque...,__label__42843 __label__41071 __label__5990 __...
5,"[atrial, fibrillation, intraventricular, condu...","[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...",atrial fibrillation intraventricular conductio...,__label__42843 __label__41071 __label__5990 __...


In [ ]:
with open("ReportTokens.txt", 'w') as f:
    dfAsString = textfiledf[["Labels","ReportTokens"]].to_string(header=False, index=False)
    f.write(dfAsString)



In [ ]:
textfiledf_pri=limited_processeddf[["TEXT","ICD9_CODE_primary"]]

textfiledf_pri["ReportTokens"]=textfiledf_pri["TEXT"].apply(lambda x: stringText(x))
textfiledf_pri["Labels"]=textfiledf_pri["ICD9_CODE_primary"].apply(lambda x: Labels(x))
textfiledf_pri.head(5)

with open("ReportTokensPri.txt", 'w') as f:
    dfAsString = textfiledf[["Labels","ReportTokens"]].to_string(header=False, index=False)
    f.write(dfAsString)


C:\Users\balle\AppData\Local\Temp/ipykernel_8176/759764125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textfiledf_pri["ReportTokens"]=textfiledf_pri["TEXT"].apply(lambda x: stringText(x))
C:\Users\balle\AppData\Local\Temp/ipykernel_8176/759764125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textfiledf_pri["Labels"]=textfiledf_pri["ICD9_CODE_primary"].apply(lambda x: Labels(x))


In [ ]:
with open("SampleReportTokensPri.txt", 'w') as f:
    dfAsString = textfiledf_pri[["Labels","ReportTokens"]].iloc[0:100].to_string(header=False, index=False)
    f.write(dfAsString)


In [ ]:
textfiledf_pri[["Labels","ReportTokens"]].iloc[0:100]

,Labels,ReportTokens
1,__label__428 __label__410 __label__599 __label...,patient test information indication aortic val...
2,__label__428 __label__410 __label__599 __label...,patient test information indication aortic val...
3,__label__428 __label__410 __label__599 __label...,sinus rhythm frequent atrial premature beats c...
4,__label__428 __label__410 __label__599 __label...,rhythm is most likely sinus rhythm with freque...
5,__label__428 __label__410 __label__599 __label...,atrial fibrillation intraventricular conductio...
...,...,...
98,__label__806 __label__584 __label__599 __label...,dddd d d d dd pm c spine non trauma w flex ext...
99,__label__806 __label__584 __label__599 __label...,dddd d d d dd pm chest pa lat clip clip number...
100,__label__806 __label__584 __label__599 __label...,dddd d d d dd pm chest portable ap clip clip n...
101,__label__806 __label__584 __label__599 __label...,dddd d d d dd am chest portable ap clip clip n...


In [ ]:
df= textfiledf_pri[["Labels","ReportTokens"]].sample(frac=1)
shuffled = textfiledf_pri[["Labels","ReportTokens"]].sample(frac=1)
result = np.array_split(shuffled, 5)  
k=0
for part in result:
    print(part.head())


                                                   Labels  \
357544  __label__962 __label__584 __label__518 __label...   
268343  __label__562 __label__518 __label__482 __label...   
177878  __label__153 __label__428 __label__997 __label...   
288991  __label__482 __label__995 __label__785 __label...   
74910   __label__250 __label__707 __label__427 __label...   

                                             ReportTokens  
357544  pmh this is a dd year old man with history of ...  
268343  shift evaluation pt is dd yo female with hx of...  
177878  atrial fibrillation diffuse non specific st t ...  
288991  demographics day of intubation day of mechanic...  
74910   temp off set d pt has oximetric swan location ...  
                                                   Labels  \
185905  __label__518 __label__507 __label__510 __label...   
187411  __label__410 __label__414 __label__428 __label...   
370923  __label__428 __label__427 __label__584 __label...   
165274  __label__441 __label_

In [ ]:
from sklearn.model_selection import KFold

#added some parameters
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
result = next(kf.split(df), None)
#print (result)

train = df.iloc[result[0]]
test =  df.iloc[result[1]]

#print (train)

k=0
for train_index, test_index in kf.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    df_train, df_test = df.iloc[train_index], df.iloc[test_index]
    
    with open("ReportTokensPriTrain"+str(k)+".txt", 'w') as f:
        dfAsString = df_train.to_string(header=False, index=False)
        f.write(dfAsString)
    
    with open("ReportTokensPriTest"+str(k)+".txt", 'w') as f:
        dfAsString = df_test.to_string(header=False, index=False)
        f.write(dfAsString)
    
    k=k+1
    
    
    




TRAIN: [     1      2      4 ... 399620 399621 399622] TEST: [     0      3     16 ... 399610 399612 399615]
TRAIN: [     0      1      2 ... 399619 399621 399622] TEST: [     9     14     18 ... 399598 399607 399620]
TRAIN: [     0      1      3 ... 399619 399620 399621] TEST: [     2      4      5 ... 399617 399618 399622]
TRAIN: [     0      1      2 ... 399618 399620 399622] TEST: [    10     12     15 ... 399616 399619 399621]
TRAIN: [     0      2      3 ... 399620 399621 399622] TEST: [     1      6      8 ... 399599 399608 399611]


In [ ]:
df= textfiledf[["Labels","ReportTokens"]].sample(frac=1)
shuffled = textfiledf[["Labels","ReportTokens"]].sample(frac=1)
result = np.array_split(shuffled, 5)  

kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
result = next(kf.split(df), None)

k=0
for train_index, test_index in kf.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    df_train, df_test = df.iloc[train_index], df.iloc[test_index]
    
    with open("ReportTokensTrain"+str(k)+".txt", 'w') as f:
        dfAsString = df_train.to_string(header=False, index=False)
        f.write(dfAsString)
    
    with open("ReportTokensTest"+str(k)+".txt", 'w') as f:
        dfAsString = df_test.to_string(header=False, index=False)
        f.write(dfAsString)
    
    k=k+1
    
    
    




TRAIN: [     1      2      4 ... 399620 399621 399622] TEST: [     0      3     16 ... 399610 399612 399615]
TRAIN: [     0      1      2 ... 399619 399621 399622] TEST: [     9     14     18 ... 399598 399607 399620]
TRAIN: [     0      1      3 ... 399619 399620 399621] TEST: [     2      4      5 ... 399617 399618 399622]
TRAIN: [     0      1      2 ... 399618 399620 399622] TEST: [    10     12     15 ... 399616 399619 399621]
TRAIN: [     0      2      3 ... 399620 399621 399622] TEST: [     1      6      8 ... 399599 399608 399611]


# Train and Test


In [ ]:
import fasttext
precisionlist=[]
recalllist=[]
F1list=[]
for kfold in range(0,5):
    model = fasttext.train_supervised("ReportTokensPriTrain"+str(kfold)+".txt", loss="ova")
    _, precision, recall = model.test("ReportTokensPriTest"+str(kfold)+".txt", k=-1, threshold=0.5)
    precisionlist.append(precision)
    print(_,precision,recall)
    recalllist.append(recall)
    F1=2*precision*recall/(precision+recall)
    F1list.append(F1)
    print(F1)

print("Average Precision:", sum(precisionlist)/5)
print("Average Recall:", sum(recalllist)/5)
print("Average F1:", sum(F1list)/5)

79925 0.7402143076911896 0.22797981199848977
0.3485952151002896
79925 0.7411325737057959 0.2286626423056291
0.34949508886903446
79925 0.7418344410876133 0.2289040447167356
0.3498550980688292
79924 0.7439531602915056 0.22740350205593668
0.34833251384105346
79924 0.7410001112739905 0.22868624761944578
0.3495079277514372
Average Precision: 0.7416269188100191
Average Recall: 0.22832724973924737
Average F1: 0.3491571687261288


In [ ]:
model.predict("atrial fibrillation diffuse non specific", k=-1, threshold=0.5)

(('__label__427', '__label__250', '__label__428', '__label__584'),
 array([1.00001001, 1.00001001, 0.62978464, 0.53899324]))

In [ ]:
precisionlist=[]
recalllist=[]
F1list=[]
for kfold in range(0,5):
    model = fasttext.train_supervised("ReportTokensTrain"+str(kfold)+".txt", loss="ova")
    _, precision, recall = model.test("ReportTokensTest"+str(kfold)+".txt", k=-1, threshold=0.5)
    precisionlist.append(precision)
    print(_,precision,recall)
    recalllist.append(recall)
    F1=2*precision*recall/(precision+recall)
    F1list.append(F1)
    print(F1)

print("Average Precision:", sum(precisionlist)/5)
print("Average Recall:", sum(recalllist)/5)
print("Average F1:", sum(F1list)/5)

79925 0.6606319555611859 0.12831199895018133
0.21488727127879093
79925 0.6606048788097421 0.1260349825593162
0.21168346143680422
79925 0.6617981775653091 0.12554973438687106
0.21105939102582214
79924 0.6612903225806451 0.12693204689469
0.21298287764323015
79924 0.6626049690160422 0.1280431301918804
0.2146138450177857
Average Precision: 0.6613860607065848
Average Recall: 0.1269743785965878
Average F1: 0.21304536928048662
